# PART 1: Data Scraping
## Connect to Reddit API

In [1]:
import praw

reddit = praw.Reddit(
    client_id="xn1EzLWHVbp2ZOuTiNYk-w",
    client_secret="rSRccp3N37yjKuQQWcPxW_SKSXe0SA",
    user_agent="<NanyangTechnologicalUniversity>-<information retrieval project>:<v1> (by /u/CitronResponsible811)",
    username="CitronResponsible811",
    password="Password123"
)

In [2]:
# Check if have read & write access
# Read access -> for requesting post & comments information
# Write access -> for requesting subreddit moderator list
reddit.user.me()

Redditor(name='CitronResponsible811')

## Documentation about reddit API crawling library
- Post docs: https://praw.readthedocs.io/en/stable/code_overview/models/submission.html#submission
- Comments docs: https://praw.readthedocs.io/en/stable/code_overview/models/comment.html

## Some errors encountered
- instances of the MoreComments class, which represent "load more comments" links on the Reddit website.

In [3]:
import csv
import emoji
import time
import os
from praw.models import MoreComments

data_dir = 'data'

def write_headers(subreddit_name):
    # Write header first
    with open(f'{os.path.join(data_dir, subreddit_name)}-posts.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'author',
            'author_flair_text',
            'clicked',
            'created_utc',
            'distinguished',
            'edited',
            'id',
            'is_original_content',
            'is_self',
            'link_flair_text',
            'locked',
            'name',
            'num_comments',
            'over_18',
            'permalink',
            'saved',
            'score',
            'selftext',
            'spoiler',
            'stickied',
            'subreddit_name',
            'title',
            'upvote_ratio',
            'url'
        ])
        
    with open(f'{os.path.join(data_dir, subreddit_name)}-comments.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'author',
            'body',
            'body_html',
            'created_utc',
            'distinguished',
            'edited',
            'id',
            'is_submitter',
            'link_id',
            'parent_id',
            'permalink',
            'saved',
            'score',
            'stickied',
            'subreddit_name',
            'subreddit_id'
        ])

In [4]:
def extract_all_post_comments_post_ids(mod_names, posts, subreddit_name, post_ids=[]):
    # Prep headers first
    write_headers(subreddit_name)
    
    for index, post in enumerate(posts, start=1):
        # Skip mod posts, usually about rules etc, not helpful at all
        if post.author and post.author.name in mod_names:
            print(f"No: {index}, Skipped mod post...")
            time.sleep(60) # Sleep for 60 seconds to avoid rate limit
            continue
        # Only write unique posts & comments
        if post.id not in post_ids:
            post_row = [
                getattr(post, 'author', '[deleted]') or '[deleted]', # Some accounts are deleted
                getattr(post, 'author_flair_text', 'None') or 'None',
                getattr(post, 'clicked', None) or 'None',
                # post.comments, # No need instance
                getattr(post, 'created_utc', None) or 'None',
                getattr(post, 'distinguished', None) or 'None',
                getattr(post, 'edited', None) or 'None',
                getattr(post, 'id', None) or 'None',
                getattr(post, 'is_original_content', None) or 'None',
                getattr(post, 'is_self', None) or 'None',
                #post.link_flair_template_id,
                getattr(post, 'link_flair_text', None) or 'None',
                getattr(post, 'locked', None) or 'None',
                getattr(post, 'name', None) or 'None',
                getattr(post, 'num_comments', None) or 'None',
                getattr(post, 'over_18', None) or 'None',
                getattr(post, 'permalink', None) or 'None',
                # post.poll_data,
                getattr(post, 'saved', None) or 'None',
                getattr(post, 'score', None) or 'None',
                getattr(post, 'selftext', None) or 'None',
                getattr(post, 'spoiler', None) or 'None',
                getattr(post, 'stickied', None) or 'None',
                subreddit_name,
                getattr(post, 'title', None) or 'None',
                getattr(post, 'upvote_ratio', None) or 'None',
                getattr(post, 'url', None) or 'None'
            ]
            print(f"No: {index}, Title: {subreddit_name}:{post.title}, PostID: {post.id}")
            
            with open(f'{os.path.join(data_dir, subreddit_name)}-posts.csv', 'a', newline='', encoding='utf-8') as f:
                # Write to file
                writer = csv.writer(f)
                writer.writerow(post_row)
                
            time.sleep(60) # Sleep for 60 seconds to avoid rate limit
    
            # Write post comments too
            post = reddit.submission(id=post_row[6]) # post.id
            
            # post.comments.replace_more(limit=None)  # Fetch all comments replies
            # for comment in post.comments.list():
            #     # Skip comments that are deleted
            #     if comment.body == '[deleted]':
            #         continue
            # If you want to fetch all comments, comment out skipping of MoreComments
            
            for comment in post.comments:
                if isinstance(comment, MoreComments):
                    # Skip lower level comments
                    continue
                # Skip mod comments, usually about rules
                if comment.author and comment.author.name in mod_names:
                    continue
                # Skip comments that are deleted
                if comment.body == '[deleted]' or comment.body == '[removed]':
                    continue

                comment_row = [
                    getattr(comment, 'author', '[deleted]') or '[deleted]', # Some accounts are deleted
                    emoji.demojize(getattr(comment, 'body', 'None')),
                    emoji.demojize(getattr(comment, 'body_html', 'None')),
                    getattr(comment, 'created_utc', None) or 'None',
                    getattr(comment, 'distinguished', None) or 'None',
                    getattr(comment, 'edited', None) or 'None',
                    getattr(comment, 'id', None) or 'None',
                    getattr(comment, 'is_submitter', None) or 'None',
                    getattr(comment, 'link_id', None) or 'None',
                    getattr(comment, 'parent_id', None) or 'None',
                    getattr(comment, 'permalink', None) or 'None',
                    # comment.replies, # No need instance, dont need replies to top level comments anyways
                    getattr(comment, 'saved', None) or 'None',
                    getattr(comment, 'score', None), # Downvotes - Upvotes
                    getattr(comment, 'stickied', None) or 'None',
                    # comment.submission, # No need instance, no need instance of user's posts
                    subreddit_name,
                    getattr(comment, 'subreddit_id', None) or None
                ]
    
                with open(f'{os.path.join(data_dir, subreddit_name)}-comments.csv', 'a', newline='', encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow(comment_row)
                    
            post_ids.append(post.id)
    return post_ids

# Get subreddit mod ids
def get_mod_list(subreddit_name):
    mod_names = []
    for moderator in reddit.subreddit(subreddit_name).moderator():
        mod_names.append(moderator.name)
        
    return mod_names

In [5]:
# Start extracting
## Specialized subreddits
specialized_subreddits = [
    # General electric car subreddits
    # "electricvehicles",
    # "electriccars",
    # "nzev",
    # "ElectricCarUK",
    # "evcharging",
    
    # Electric Car brands subreddits
    # "teslamotors", # Tesla 100% electric car brand
    # "RealTesla",
    # "BMWi3", # BMW I3 car model electric
    # "leaf", # Nissan LEAF EV car model
    # "BoltEV", # Chevrolet Bolt EV car model
    # "F150Lightning", # Ford F150 Lightning EV car model
    # "Taycan", # Porsche Taycan EV car model
    # "KiaEV6", # Kia EV6 car model
]

for subreddit_name in specialized_subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.top(limit=100, time_filter='all')
    extract_all_post_comments_post_ids(get_mod_list(subreddit_name), posts, subreddit_name)

No: 1, Title: F150Lightning:Cybertruck broke at King of Hammers, PostID: 1ahvzms
No: 2, Title: F150Lightning:No more charging while at work, PostID: 18wq76i
No: 3, Title: F150Lightning:Buyback complete, surrendered it back to ford today, goodbye lightning ;(, PostID: 19elr6g
No: 4, Title: F150Lightning:The worst part of owning an EV is… Electrify America, PostID: 16nxgez
No: 5, Title: F150Lightning:Ford F-150 Lightning breaks monthly sales record, doubling in November, PostID: 188rf2m
No: 6, Title: F150Lightning:Cybertruck and Lightning comparison, PostID: 17ow957
No: 7, Title: F150Lightning:My first EV. Picked up a Lariant, PostID: 1b8t9aa
No: 8, Title: F150Lightning:Who got me beat? 2023 Lariat ER for $49,215., PostID: 1b3irew
No: 9, Title: F150Lightning:Any smoke around this being true?, PostID: 1au5ppz
No: 10, Title: F150Lightning:Ford U.S. EV Sales Surged 81% In February 2024, PostID: 1b6q9yy
No: 11, Title: F150Lightning:'Ford Cutting F-150 Lightning Production By Half In 2024, Su

## General subreddits with search filters

In [ ]:
general_subreddits = [
    "cars",
]

search_terms = [
    'ev',
]

for subreddit_name in general_subreddits:
    post_ids = []
    for search_term in search_terms:
        subreddit = reddit.subreddit(subreddit_name)
        search_results = subreddit.search(search_term, sort='relevance', limit=100)
        post_ids.extend(extract_all_post_comments_post_ids(get_mod_list(subreddit_name), search_results, f'{subreddit_name}-{search_term}', post_ids))
        post_ids = list(set(post_ids))

# PART 2: Get dataset summaries
## Summary of all different subreddit posts & comments

In [9]:
# Load all comments file, check unique word count
import os
import pandas as pd

data_dir = 'data'

# Since same format, combine all comments csv into a massive df
comment_main_df = pd.DataFrame()
post_main_df = pd.DataFrame()
for file in os.listdir(data_dir):
    df = pd.read_csv(f'{data_dir}/{file}')
    print(f"File: {file}, Rows: {len(df)}")
    if file.endswith("comments.csv"):
        comment_main_df = pd.concat([comment_main_df, df])
    elif file.endswith("posts.csv"):
        post_main_df = pd.concat([post_main_df, df])

File: BMWi3-comments.csv, Rows: 1268
File: BMWi3-posts.csv, Rows: 100
File: BoltEV-comments.csv, Rows: 4439
File: BoltEV-posts.csv, Rows: 98
File: electriccars-comments.csv, Rows: 2303
File: electriccars-posts.csv, Rows: 99
File: ElectricCarUK-comments.csv, Rows: 73
File: ElectricCarUK-posts.csv, Rows: 55
File: electricvehicles-comments.csv, Rows: 6882
File: electricvehicles-posts.csv, Rows: 100
File: evcharging-comments.csv, Rows: 1339
File: evcharging-posts.csv, Rows: 96
File: F150Lightning-comments.csv, Rows: 3419
File: F150Lightning-posts.csv, Rows: 99
File: KiaEV6-comments.csv, Rows: 1602
File: KiaEV6-posts.csv, Rows: 100
File: leaf-comments.csv, Rows: 1515
File: leaf-posts.csv, Rows: 99
File: nzev-comments.csv, Rows: 1446
File: nzev-posts.csv, Rows: 83
File: RealTesla-comments.csv, Rows: 10786
File: RealTesla-posts.csv, Rows: 100
File: Taycan-comments.csv, Rows: 1334
File: Taycan-posts.csv, Rows: 100
File: teslamotors-comments.csv, Rows: 11788
File: teslamotors-posts.csv, Rows: 1

## Combined size

In [10]:
post_main_df.shape

(1229, 24)

In [11]:
comment_main_df.shape

(48194, 16)

## Checking for duplicated id & remove them

In [12]:
# Check for duplicates using id
post_main_df[post_main_df.duplicated(subset=['id'], keep='first')]

,author,author_flair_text,clicked,created_utc,distinguished,edited,id,is_original_content,is_self,link_flair_text,...,permalink,saved,score,selftext,spoiler,stickied,subreddit_name,title,upvote_ratio,url


In [13]:
comment_main_df[comment_main_df.duplicated(subset=['id'], keep='first')]

,author,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_name,subreddit_id


In [14]:
# Remove any duplicates detected
post_main_df.drop_duplicates(subset=['id'], keep='first')
comment_main_df.drop_duplicates(subset=['id'], keep='first')

,author,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_name,subreddit_id
0,McNutWaffle,Congrats on both! Glad I'm not the only who w...,"<div class=""md""><p>Congrats on both! Glad I&#...",1.648494e+09,NaN,NaN,i2h01oy,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,13,NaN,BMWi3,t5_2y1ho
1,pboksz,Congrats! That is one impressive reward! This ...,"<div class=""md""><p>Congrats! That is one impre...",1.648494e+09,NaN,NaN,i2h052j,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,10,NaN,BMWi3,t5_2y1ho
2,JealousPomegranate62,Congratulations!!! It’s beautiful !!!,"<div class=""md""><p>Congratulations!!! It’s bea...",1.648504e+09,NaN,NaN,i2hnwdm,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,4,NaN,BMWi3,t5_2y1ho
3,MMinthemirror,Congratulations! That's a fantastic car for a ...,"<div class=""md""><p>Congratulations! That&#39;s...",1.648504e+09,NaN,NaN,i2hmqoo,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,3,NaN,BMWi3,t5_2y1ho
4,janithemuse488,Welcome to the family! Congratulations.,"<div class=""md""><p>Welcome to the family! Con...",1.648509e+09,NaN,NaN,i2hy1od,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,3,NaN,BMWi3,t5_2y1ho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11783,mariscvaco,Fake Email,"<div class=""md""><p>Fake Email</p>\r\n</div>",1.654051e+09,NaN,NaN,iaqnmqn,NaN,t3_v25ee0,t3_v25ee0,/r/teslamotors/comments/v25ee0/tesla_is_no_lon...,NaN,-23,NaN,teslamotors,t5_2s3j5
11784,[deleted],fake,"<div class=""md""><p>fake</p>\r\n</div>",1.654056e+09,NaN,NaN,iaqwyap,NaN,t3_v25ee0,t3_v25ee0,/r/teslamotors/comments/v25ee0/tesla_is_no_lon...,NaN,-33,NaN,teslamotors,t5_2s3j5
11785,shitaki13,Looks fake. Why would you have to block Elon’s...,"<div class=""md""><p>Looks fake. Why would you h...",1.654053e+09,NaN,NaN,iaqs6p4,NaN,t3_v25ee0,t3_v25ee0,/r/teslamotors/comments/v25ee0/tesla_is_no_lon...,NaN,-23,NaN,teslamotors,t5_2s3j5
11786,jrr6415sun,That’s an email sent to execs though. It doesn...,"<div class=""md""><p>That’s an email sent to exe...",1.654053e+09,NaN,NaN,iaqt5uv,NaN,t3_v25ee0,t3_v25ee0,/r/teslamotors/comments/v25ee0/tesla_is_no_lon...,NaN,-15,NaN,teslamotors,t5_2s3j5


## Checking for duplicated comments & post (Some might be crossposted, different id but same title but different comments)
- We decided to keep them as they are same post contents but different comments

In [15]:
post_main_df[post_main_df.duplicated(subset=['title'], keep=False)]

,author,author_flair_text,clicked,created_utc,distinguished,edited,id,is_original_content,is_self,link_flair_text,...,permalink,saved,score,selftext,spoiler,stickied,subreddit_name,title,upvote_ratio,url
20,wewewawa,NaN,NaN,1.699695e+09,NaN,NaN,17sr2l7,NaN,NaN,NaN,...,/r/electriccars/comments/17sr2l7/if_you_bought...,NaN,125,NaN,NaN,NaN,electriccars,If You Bought A Tesla One Year Ago It Has Lost...,0.82,https://jalopnik.com/if-you-bought-a-tesla-one...
34,manual_tranny,NaN,NaN,1.691671e+09,NaN,NaN,15nbggi,NaN,NaN,News,...,/r/electricvehicles/comments/15nbggi/disapprov...,NaN,2032,NaN,NaN,NaN,electricvehicles,Disapproval of Elon Musk is top reason Tesla o...,0.91,https://electrek.co/2023/07/27/disapproval-elo...
19,eepluribus,NaN,NaN,1.664449e+09,NaN,NaN,xr4lfq,NaN,NaN,NaN,...,/r/evcharging/comments/xr4lfq/using_ev6s_v2l_t...,NaN,55,NaN,NaN,NaN,evcharging,Using EV6's V2L to power the essentials during...,1.00,https://v.redd.it/564kacodtrq91
59,capt-ramius,NaN,NaN,1.708554e+09,NaN,NaN,1awpglj,NaN,NaN,NaN,...,/r/evcharging/comments/1awpglj/tesla_scnacs_ad...,NaN,31,NaN,NaN,NaN,evcharging,Tesla SC/NACS Adapter Appeared on Ford Parts W...,0.95,https://i.redd.it/fqsmcgd8a0kc1.jpeg
24,capt-ramius,‘22 SR XLT 312A,NaN,1.708551e+09,NaN,NaN,1awntal,NaN,NaN,NaN,...,/r/F150Lightning/comments/1awntal/tesla_scnacs...,NaN,312,https://x.com/eric_l_roe/status/17603712194956...,NaN,NaN,F150Lightning,Tesla SC/NACS Adapter Appeared on Ford Parts W...,0.97,https://i.redd.it/fqsmcgd8a0kc1.jpeg
11,eepluribus,EV6 Wind,NaN,1.664446e+09,NaN,NaN,xr3qt0,NaN,NaN,NaN,...,/r/KiaEV6/comments/xr3qt0/using_ev6s_v2l_to_po...,NaN,137,NaN,NaN,NaN,KiaEV6,Using EV6's V2L to power the essentials during...,0.99,https://v.redd.it/564kacodtrq91
5,mananuku,NaN,NaN,1.706079e+09,NaN,NaN,19eb37j,NaN,NaN,NaN,...,/r/nzev/comments/19eb37j/admin_fees_are_worse_...,NaN,137,$13 admin fee for RUC’s is just taking the piss.,NaN,NaN,nzev,Admin fees are worse than ticketek,0.88,https://i.redd.it/di3oqv5q5cec1.jpeg
10,Local_Yoghurt9320,NaN,NaN,1.710825e+09,NaN,NaN,1bicfpb,NaN,NaN,NaN,...,/r/nzev/comments/1bicfpb/admin_fees_are_worse_...,NaN,124,NaN,NaN,NaN,nzev,Admin fees are worse than ticketek,0.92,https://i.redd.it/se96alh148pc1.jpeg
5,gman1023,NaN,NaN,1.690573e+09,NaN,NaN,15c6xk9,NaN,NaN,HELP NEEDED,...,/r/RealTesla/comments/15c6xk9/disapproval_of_e...,NaN,5031,NaN,NaN,NaN,RealTesla,Disapproval of Elon Musk is top reason Tesla o...,0.95,https://electrek.co/2023/07/27/disapproval-elo...
77,TiltedWit,NaN,NaN,1.699650e+09,NaN,NaN,17sdqni,NaN,NaN,NaN,...,/r/RealTesla/comments/17sdqni/if_you_bought_a_...,NaN,1405,NaN,NaN,NaN,RealTesla,If You Bought A Tesla One Year Ago It Has Lost...,0.94,https://jalopnik.com/if-you-bought-a-tesla-one...


In [16]:
comment_main_df[comment_main_df.duplicated(subset=['body'], keep=False)]

,author,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_name,subreddit_id
6,JesseWebDotCom,Congrats,"<div class=""md""><p>Congrats</p>\n</div>",1.648558e+09,NaN,NaN,i2k2yt1,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,3,NaN,BMWi3,t5_2y1ho
34,Tide865,Congrats!,"<div class=""md""><p>Congrats!</p>\n</div>",1.708648e+09,NaN,NaN,krowqok,NaN,t3_1axmm77,t3_1axmm77,/r/BMWi3/comments/1axmm77/100k_miles/krowqok/,NaN,3,NaN,BMWi3,t5_2y1ho
70,Darenhayes1978,Congrats!,"<div class=""md""><p>Congrats!</p>\n</div>",1.685016e+09,NaN,NaN,jljyc47,NaN,t3_13rc663,t3_13rc663,/r/BMWi3/comments/13rc663/picked_her_up_straig...,NaN,2,NaN,BMWi3,t5_2y1ho
92,Rough-Friendship2657,Beautiful!,"<div class=""md""><p>Beautiful!</p>\n</div>",1.684768e+09,NaN,NaN,jl5xmy6,NaN,t3_13o1009,t3_13o1009,/r/BMWi3/comments/13o1009/picked_up_this_littl...,NaN,2,NaN,BMWi3,t5_2y1ho
105,Disavowed_Rogue,Looks great!,"<div class=""md""><p>Looks great!</p>\n</div>",1.696521e+09,NaN,NaN,k3l86qm,NaN,t3_170lc98,t3_170lc98,/r/BMWi3/comments/170lc98/pulled_the_ev_trigge...,NaN,5,NaN,BMWi3,t5_2y1ho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11654,twinbee,Well played.,"<div class=""md""><p>Well played.</p>\r\n</div>",1.569799e+09,NaN,NaN,f1xk1mf,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,3,NaN,teslamotors,t5_2s3j5
11656,warboar,Amazing,"<div class=""md""><p>Amazing</p>\r\n</div>",1.569808e+09,NaN,NaN,f1y1r70,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,3,NaN,teslamotors,t5_2s3j5
11676,zult1,Omg this is amazing!,"<div class=""md""><p>Omg this is amazing!</p>\r\...",1.569814e+09,NaN,NaN,f1ycxr6,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,1,NaN,teslamotors,t5_2s3j5
11682,dflame45,This is hilarious,"<div class=""md""><p>This is hilarious</p>\r\n</...",1.569819e+09,NaN,NaN,f1ylkcu,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,1,NaN,teslamotors,t5_2s3j5


## Save corpus to csv

In [17]:
with open(f'{os.path.join(data_dir, "all-posts.csv")}', 'w', newline='', encoding='utf-8') as f:
    post_main_df.to_csv(f, index=False)
    
with open(f'{os.path.join(data_dir, "all-comments.csv")}', 'w', newline='', encoding='utf-8') as f:
    comment_main_df.to_csv(f, index=False)